# Theano

Based somewhat on http://docs.pymc.io/theano.html

In [2]:
import theano
import theano.tensor as tt
import numpy as np

In [3]:
a = tt.scalar('a')
x = tt.vector('x')
y = tt.ivector('y')

inner = a * x**3 + y**2
out = tt.exp(inner).sum()

In [4]:
func = theano.function([a, x, y], [out])

In [38]:
a_val = 1.2
x_vals = np.random.randn(100)
y_vals = (np.random.randn(100) * 2).astype(np.int)

func(a_val, x_vals, y_vals)

[array(129318357309.86845)]

In [39]:
def np_func(a, x, y):
    inner = a * x**3 + y**2
    return np.sum(np.exp(inner))

np_func(a_val, x_vals, y_vals)

129318357309.86838

In [40]:
%timeit(np_func(a_val, x_vals, y_vals))

21.6 µs ± 310 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [41]:
%timeit(func(a_val, x_vals, y_vals))

33.6 µs ± 1.88 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


So from a speed point of view, this is a little disappointing.  They are about equally fast with 1000 terms, and the theano version starts to win at 10000 size arrays.

In [42]:
a = tt.vector('a')
b = tt.switch((a > 0).all(), tt.sqrt(a), -a)

In [43]:
func = theano.function([a], [b])

In [48]:
def np_func(a):
    a = np.asarray(a)
    if np.all(a>0):
        return np.sqrt(a)
    return -a

In [49]:
func([1,2,3,4]), np_func([1,2,3,4])

([array([ 1.        ,  1.41421356,  1.73205081,  2.        ])],
 array([ 1.        ,  1.41421356,  1.73205081,  2.        ]))

In [50]:
func([1,-2,3,4]), np_func([1,-2,3,4])

([array([-1.,  2., -3., -4.])], array([-1,  2, -3, -4]))

In [54]:
a = np.random.random(1000000) - 0.5
%timeit(func(a))
%timeit(np_func(a))

7.08 ms ± 66.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
4.23 ms ± 19.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [55]:
a = tt.vector('a')
b = tt.set_subtensor(a[:10], 1)

In [56]:
func = theano.function([a], [b])

In [60]:
def np_func1(a):
    b = a.copy()
    b[:10] = 1
    return b

def np_func2(a):
    b = np.empty_like(a)
    b[:10] = 1
    b[10:] = a[10:]
    return b

a = np.asarray(list(range(20)), dtype=np.float)
np_func2(a)

array([  1.,   1.,   1.,   1.,   1.,   1.,   1.,   1.,   1.,   1.,  10.,
        11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.])

In [61]:
a = np.asarray(list(range(20)), dtype=np.float)
%timeit(func(a))
%timeit(np_func1(a))
%timeit(np_func2(a))

15.2 µs ± 111 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
1.77 µs ± 27.4 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
2.54 µs ± 53.3 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
